In [2]:
import chess
from chess.pgn import read_game
import numpy as np
from anytree import Node, RenderTree
from anytree.iterators import LevelOrderIter
from network import ChessNeuralNetwork

def board_generator():
    pgn = open("millionbase-2.22.pgn")
    while True:
        game = read_game(pgn)
        if game and len(list(game.main_line()))>0:
            node = game
            move_number = np.random.randint(0, high=len(list(game.main_line())) - 1) #don't take the last move
            while 2*(node.board().fullmove_number-1)+int(not node.board().turn) < move_number:
                next_node = node.variation(0)
                node = next_node
            yield node.board()
        else:
            pgn.seek(0)

In [3]:
gen = board_generator()

In [4]:
b = gen.__next__()

In [5]:
def build_tree(root, depth):
    if depth > 0:
        board = root.board
        legal_moves = board.legal_moves
        nodes = []
        for legal_move in legal_moves:
            child_board = board.copy()
            child_board.push(legal_move)
            node = Node(str(legal_move), parent=root, board=child_board, move=legal_move)
            nodes.append(node)
            build_tree(node, depth-1)

In [6]:
root = Node('root', board=chess.Board(), move=None)
build_tree(root, 2)
l=[node for node in LevelOrderIter(root)]

In [7]:
def simple_value(board):
    values = [1, 3, 3, 5, 9]
    s = 0
    for i, v in enumerate(values):
        s += ChessNeuralNetwork.pad_bitmask(board.pieces_mask(i + 1, 1)).sum() * v
        s -= ChessNeuralNetwork.pad_bitmask(board.pieces_mask(i + 1, 0)).sum() * v
    return s

In [8]:
def alpha_beta(node, depth, alpha, beta):
    if depth == 0 or node.board.is_game_over():
        return simple_value(node.board)
    
    legal_moves = list(node.board.legal_moves)
    child_boards = [node.board.copy() for _ in legal_moves]
    children = []
    for idx in range(len(node.board.legal_moves)):
        child_boards[idx].push(legal_moves[idx])
        child = Node(str(legal_moves[idx]), parent=node, board=child_boards[idx], move=legal_moves[idx])
        children.append(child)
    print('num children:', len(children))
    n = None
    if node.board.turn:
        v = -100000
        for child in children:
            print(child)
            print(depth)
            vv, nn = alpha_beta(child, depth - 1, alpha, beta)
            print(vv)
            if vv > v:
                v = vv
                n = nn
            alpha = max(alpha, v)
            if beta <= alpha:
                break  # (* β cut-off *)
        return v, n
    else:
        v = 100000
        for child in children:
            print(child)
            print(depth)
            vv, nn = alpha_beta(child, depth - 1, alpha, beta)
            print(vv)
            if vv < v:
                v = vv
                n = nn
            beta = min(beta, v)
            if beta <= alpha:
                break  # (* α cut-off *)
        return v, n

In [9]:
b = gen.__next__()
root = Node('root', board=b, move=None)
alpha_beta(root, 2, -100000, 100000)

num children: 20
Node('/root/b8a6', board=Board('r1bqkbnr/pppppppp/n7/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 1 2'), move=Move.from_uci('b8a6'))
2
num children: 28
Node('/root/b8a6/b1d2', board=Board('r1bqkbnr/pppppppp/n7/8/3P4/8/PPPNPPPP/R1BQKBNR b KQkq - 2 2'), move=Move.from_uci('b1d2'))
1


TypeError: 'numpy.int64' object is not iterable

In [10]:
alpha_beta(root, 2, -100000, 100000)

num children: 20
Node('/root/b8a6', board=Board('r1bqkbnr/pppppppp/n7/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 1 2'), move=Move.from_uci('b8a6'))
2
num children: 28
Node('/root/b8a6/b1d2', board=Board('r1bqkbnr/pppppppp/n7/8/3P4/8/PPPNPPPP/R1BQKBNR b KQkq - 2 2'), move=Move.from_uci('b1d2'))
1


TypeError: 'numpy.int64' object is not iterable

In [13]:
b = gen.__next__()
root = Node('root', board=b, move=None)
build_tree(root, 2)
print(alpha_beta(root, 2, -100000, 100000))
print(alpha_beta_board(b, 2, -100000, 100000))

num children: 28
Node('/root/d6h2', board=Board('b1rr4/p7/5k1p/2pp2p1/8/1NP5/P3RPPb/2B1R1K1 w - - 0 26'), move=Move.from_uci('d6h2'))
2
num children: 3
Node('/root/d6h2/g1f1', board=Board('b1rr4/p7/5k1p/2pp2p1/8/1NP5/P3RPPb/2B1RK2 b - - 1 26'), move=Move.from_uci('g1f1'))
1


TypeError: 'numpy.int64' object is not iterable

In [14]:
def alpha_beta(node, depth, alpha, beta):
    if depth == 0 or node.board.is_game_over():
        return simple_value(node.board), node

    legal_moves = list(node.board.legal_moves)
    child_boards = [node.board.copy() for _ in legal_moves]
    children = []
    for idx in range(len(node.board.legal_moves)):
        child_boards[idx].push(legal_moves[idx])
        child = Node(str(legal_moves[idx]), parent=node, board=child_boards[idx], move=legal_moves[idx])
        children.append(child)
    n = node
    if node.board.turn:
        v = -100000
        for child in children:
            vv, nn = alpha_beta(child, depth - 1, alpha, beta)
            if vv > v:
                v = vv
                n = nn
            alpha = max(alpha, v)
            if beta <= alpha:
                break  # (* β cut-off *)
        return v, n
    else:
        v = 100000
        for child in children:
            vv, nn = alpha_beta(child, depth - 1, alpha, beta)
            if vv < v:
                v = vv
                n = nn
            beta = min(beta, v)
            if beta <= alpha:
                break  # (* α cut-off *)
        return v, n

In [18]:
b = gen.__next__()
root = Node('root', board=b, move=None)
build_tree(root, 2)
v, n =alpha_beta(root, 2, -100000, 100000)
print(v,n)

0 Node('/root/d1a4/b7b5', board=Board('rnbqkb1r/p3pppp/5n2/1p1p4/Q1PP4/8/PP3PPP/RNB1KBNR w KQkq - 0 6'), move=Move.from_uci('b7b5'))


In [32]:
n.path[1].move in b.legal_moves

True